# Types and Dispatch in Julia

One of the most important goals of high-level languages is to provide *polymorphism*: the ability for the same code to operate on different kinds of values.

Julia uses a vocabulary of *types* for this purpose. Types play the following roles:

- Describe "what kind of thing is this"
- Describe the representation of a value
- Driving *dispatch*: selecting one of several pieces of code
- Driving *specialization*: code is optimized by assuming values have certain types

## Describing values

In [1]:
typeof(3)

Int64

In [2]:
sizeof(Int64)

8

In [3]:
Int64.size

8

In [4]:
isbits(Int64)

false

In [5]:
Int64.mutable

false

In [6]:
supertype(Int64)

Signed

In [7]:
supertype(Signed)

Integer

In [8]:
supertype(Integer)

Real

In [9]:
supertype(Real)

Number

In [10]:
supertype(Number)

Any

In [11]:
# The subtype operator/relation
Integer <: Real

true

In [12]:
String <: Real

false

In [13]:
Any >: String

true

In [14]:
# The `isa` operator/relation
1 isa Int

true

In [15]:
1 isa String

false

Julia has roughly 5 kinds of types. We just saw two:

1. Data types - describing concrete data objects
2. Abstract types - group those together

There are three more:

1. Union types
2. UnionAll types
3. The empty type

## Union types

Expresses a *set union* of types.

In [16]:
1 isa Union{Int,String}

true

In [17]:
"hi" isa Union{Int,String}

true

## UnionAll types

Expresses an *iterated set union* of types.

In [18]:
[1] isa Vector{Int}

true

In [19]:
[1] isa (Vector{T} where T<:Real)

true

$\bigcup\limits_{T<:Real} \tt{Vector}\{T\}$

In [20]:
Union{Vector{Any},Vector{Real}} <: Vector{T} where T>:Real

true

In [21]:
T where T<:Real

Real

In [22]:
rand(1:10,2,2)

2×2 Array{Int64,2}:
 6  10
 1   2

In [25]:
?dump

search: dump



```
dump(x; maxdepth=8)
```

Show every part of the representation of a value. The depth of the output is truncated at `maxdepth`.

# Examples

```jldoctest
julia> struct MyStruct
           x
           y
       end

julia> x = MyStruct(1, (2,3));

julia> dump(x)
MyStruct
  x: Int64 1
  y: Tuple{Int64,Int64}
    1: Int64 2
    2: Int64 3

julia> dump(x; maxdepth = 1)
MyStruct
  x: Int64 1
  y: Tuple{Int64,Int64}
```


In [26]:
dump(Array)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Union{}
    ub: Any
  body: UnionAll
    var: TypeVar
      name: Symbol N
      lb: Union{}
      ub: Any
    body: Array{T,N} <: DenseArray{T,N}


In [27]:
Vector

Array{T,1} where T

In [28]:
Vector{Int} <: Vector

true

In [29]:
Vector <: Array

true

In [30]:
Vector{Int} <: Vector{Any}

false

In [31]:
typeintersect((Array{T} where T<:Real), (Array{T,2} where T>:Int))

Array{T,2} where Int64<:T<:Real

In [32]:
[[2]] isa (Vector{T} where T<:Vector{S} where S<:Integer)

true

## The empty type

Corresponds to the empty set.

In [33]:
1 isa Union{}

false

In [34]:
Union{} <: Int

true

In [35]:
Union{} <: String

true

In [36]:
Union{} <: Array

true

This represents situations where there can't be any value; e.g. an exception is thrown or the program doesn't terminate.

## Dispatch

In [37]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [38]:
methods(f)

# 5 methods for generic function "f":
[1] f(a::Integer, b::Integer) in Main at In[37]:5
[2] f(a::Number, b::Number) in Main at In[37]:2
[3] f(a::Number, b) in Main at In[37]:3
[4] f(a, b::Number) in Main at In[37]:4
[5] f(a, b) in Main at In[37]:1

In [39]:
f(1.5, 2)

"a and b are both numbers"

In [40]:
f(1, "string")

"a is a number"

In [41]:
f(1, 2)

"a and b are both integers"

In [42]:
f(1, 2, 3)

MethodError: MethodError: no method matching f(::Int64, ::Int64, ::Int64)
Closest candidates are:
  f(::Integer, ::Integer) at In[37]:5
  f(::Number, ::Number) at In[37]:2
  f(::Number, ::Any) at In[37]:3
  ...

## Tuples

A tuple is an immutable container of any combination of values.

Often used to represent e.g. ordered pairs, or for returning "multiple" values from functions.

In [43]:
t = (1, "hi", 0.33, pi)

(1, "hi", 0.33, π)

In [44]:
t[2]

"hi"

In [45]:
# "destructuring"
a, b, c = t

(1, "hi", 0.33, π)

In [46]:
a

1

In [47]:
b

"hi"

In [48]:
typeof(t)

Tuple{Int64,String,Float64,Irrational{:π}}

Tuple types represent the arguments to a function.

In [51]:
first(methods(f))

f(a::Integer, b::Integer) in Main at In[37]:5

In [49]:
first(methods(f)).sig

Tuple{typeof(f),Integer,Integer}

For every function call, the method that gets called is the most specific one such that the argument tuple type is a subtype of the signature.

## "Diagonal" dispatch

In [56]:
d(x::T, y::T) where {T} = "same type"
d(x, y) = "different types"

d (generic function with 2 methods)

In [57]:
d(1, 1)

"same type"

In [58]:
d(1, 2.0)

"different types"

In [61]:
[ m.sig for m in methods(d) ]

2-element Array{Type,1}:
 Tuple{typeof(d),T,T} where T
 Tuple{typeof(d),Any,Any}

## Variadic (or varargs) methods

In [62]:
v(x...) = (x, "zero or more")

v (generic function with 1 method)

In [63]:
v(x, xs...) = (xs, "one or more")

v (generic function with 2 methods)

In [64]:
v()

((), "zero or more")

In [65]:
v(1)

((), "one or more")

In [66]:
v(1, 2, 3, 4)

((2, 3, 4), "one or more")

## Variadic tuple types

In [67]:
foo(a::Array, Is::Int...) = 0

foo (generic function with 1 method)

In [68]:
first(methods(foo)).sig

Tuple{typeof(foo),Array,Vararg{Int64,N} where N}

In [69]:
vt = Tuple{Array, Vararg{Int}}

Tuple{Array,Vararg{Int64,N} where N}

In [70]:
isa(([1],1,2,3), vt)

true

In [71]:
isa(([1],1,0.02,3), vt)

false

## Specialization in action

Internally, the compiler generates specializations for particular types.

Example: For a 3-argument function `f`, the compiler might decide to generate a specialization for `Tuple{Int, Any, Int}`, if for some reason the second argument isn't important.

In [5]:
addall(t) = +(t...)  # "splat"

addall (generic function with 1 method)

In [4]:
?+

search: +



```
+(x, y...)
```

Addition operator. `x+y+z+...` calls this function with all arguments, i.e. `+(x, y, z, ...)`.

# Examples

```jldoctest
julia> 1 + 20 + 4
25

julia> +(1, 20, 4)
25
```

---

```
dt::Date + t::Time -> DateTime
```

The addition of a `Date` with a `Time` produces a `DateTime`. The hour, minute, second, and millisecond parts of the `Time` are used along with the year, month, and day of the `Date` to create the new `DateTime`. Non-zero microseconds or nanoseconds in the `Time` type will result in an `InexactError` being thrown.


In [3]:
a=[1,2,3,4]
addall(a)

10

In [6]:
@code_typed addall((1,2))

CodeInfo(
1 ─ %1 = (getfield)(t, 1)::Int64
│   %2 = (getfield)(t, 2)::Int64
│   %3 = Base.add_int(%1, %2)::Int64
└──      return %3
) => Int64

In [7]:
@code_typed addall((1,2,3))

CodeInfo(
1 ─ %1 = (getfield)(t, 1)::Int64
│   %2 = (getfield)(t, 2)::Int64
│   %3 = (getfield)(t, 3)::Int64
│   %4 = Base.add_int(%1, %2)::Int64
│   %5 = Base.add_int(%4, %3)::Int64
└──      return %5
) => Int64

In [8]:
function alltrue(f, itr)
    @inbounds for x in itr
        f(x) || return false
    end
    return true
end

alltrue (generic function with 1 method)

In [9]:
@which isinteger(1)

isinteger(x::Integer) in Base at number.jl:20

In [10]:
@code_typed alltrue(isinteger, [1,2,3])

CodeInfo(
1 ── %1  = Base.bitcast(UInt64, 1)::UInt64
│    %2  = Base.bitcast(UInt64, 1)::UInt64
│    %3  = Base.sub_int(%1, %2)::UInt64
│    %4  = Base.arraylen(itr)::Int64
│    %5  = Base.sle_int(0, %4)::Bool
│    %6  = Base.bitcast(UInt64, %4)::UInt64
│    %7  = Base.ult_int(%3, %6)::Bool
│    %8  = Base.and_int(%5, %7)::Bool
└───       goto #3 if not %8
2 ──       Base.arrayref(false, itr, 1)::Int64
│    %11 = Base.add_int(1, 1)::Int64
└───       goto #4
3 ──       goto #4
4 ┄─ %14 = φ (#2 => false, #3 => true)::Bool
│    %15 = φ (#2 => %11)::Int64
└───       goto #5
5 ── %17 = Base.not_int(%14)::Bool
└───       goto #11 if not %17
6 ┄─ %19 = φ (#5 => %15, #10 => %33)::Int64
│    %20 = Base.bitcast(UInt64, %19)::UInt64
│    %21 = Base.bitcast(UInt64, 1)::UInt64
│    %22 = Base.sub_int(%20, %21)::UInt64
│    %23 = Base.arraylen(itr)::Int64
│    %24 = Base.sle_int(0, %23)::Bool
│    %25 = Base.bitcast(UInt64, %23)::UInt64
│    %26 = Base.ult_int(%22, %25)::Bool
│    %27 = Base.and_int

In [11]:
@code_llvm alltrue(isinteger, [1,2,3])


;  @ In[8]:2 within `alltrue'
; Function Attrs: uwtable
define i8 @julia_alltrue_18099(%jl_value_t addrspace(10)* nonnull align 16 dereferenceable(40)) #0 {
top:
;  @ In[8]:5 within `alltrue'
  ret i8 1
}


## What is the difference between @code_native, @code_typed and @code_llvm in Julia?
https://stackoverflow.com/questions/43453944/what-is-the-difference-between-code-native-code-typed-and-code-llvm-in-julia

## Dispatch, specialization, and performance

Dynamic dispatch is traditionally considered "slow".

Instead of a `call` instruction, you need to do a table lookup procedure first.

However:
1. If types are known, the call target can be looked up at compile time.
2. The cost of dynamic dispatch is well worth it *if* you're dispatching to an optimized kernel.

## What to specialize on?

We can't specialize on *everything* because it would take too long and generate too much code.

There's no fully general and automatic approach.

We specialize on types. That's a reasonable default. If the default's not good enough, move more information into types!

A classic: specializing on the value of an integer.

In [1]:
function sum1n(::Val{N}) where {N}    # given `struct Val{N} end`
    s = 0
    for i = 1:N
        s += i
    end
    return s
end

sum1n (generic function with 1 method)

In [4]:
?where

search:



```
where
```

The `where` keyword creates a type that is an iterated union of other types, over all values of some variable. For example `Vector{T} where T<:Real` includes all [`Vector`](@ref)s where the element type is some kind of `Real` number.

The variable bound defaults to [`Any`](@ref) if it is omitted:

```julia
Vector{T} where T    # short for `where T<:Any`
```

Variables can also have lower bounds:

```julia
Vector{T} where T>:Int
Vector{T} where Int<:T<:Real
```

There is also a concise syntax for nested `where` expressions. For example, this:

```julia
Pair{T, S} where S<:Array{T} where T<:Number
```

can be shortened to:

```julia
Pair{T, S} where {T<:Number, S<:Array{T}}
```

This form is often found on method signatures.

Note that in this form, the variables are listed outermost-first. This matches the order in which variables are substituted when a type is "applied" to parameter values using the syntax `T{p1, p2, ...}`.


In [3]:
?Val

search: Val values valtype eval evalpoly evalfile @eval isvalid @evalpoly



```
Val(c)
```

Return `Val{c}()`, which contains no run-time data. Types like this can be used to pass the information between functions through the value `c`, which must be an `isbits` value. The intent of this construct is to be able to dispatch on constants directly (at compile time) without having to test the value of the constant at run time.

# Examples

```jldoctest
julia> f(::Val{true}) = "Good"
f (generic function with 1 method)

julia> f(::Val{false}) = "Bad"
f (generic function with 2 methods)

julia> f(Val(true))
"Good"
```


In [5]:
sum1n(Val{10}())

55

In [6]:
@code_llvm sum1n(Val{10}())


;  @ In[1]:2 within `sum1n'
; Function Attrs: uwtable
define i64 @julia_sum1n_17908() #0 {
top:
;  @ In[1]:6 within `sum1n'
  ret i64 55
}


In [7]:
sum1n(n::Integer) = sum1n(Val{n}())

sum1n (generic function with 2 methods)

In [8]:
sum1n(20)

210

In [9]:
sum1n(rand(1:100))  # dynamic dispatch to specialized code

2145

# "Stupid Dispatch Tricks"

## Trick 1: processing arguments recursively

The compiler's optimizations can be exploited to move parts of your own computations to compile time (thus saving time at run time). The general idea is to represent more information within types, instead of using values.

Example: drop the first element of a tuple.

In [10]:
tuple_tail1(t) = t[2:end]

tuple_tail1 (generic function with 1 method)

In [11]:
tuple_tail1((1,2,"hi"))

(2, "hi")

In [12]:
@code_typed tuple_tail1((1,2,"hi"))

CodeInfo(
1 ── %1  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{Any,1}, svec(Any, Int64), 0, :(:ccall), Array{Any,1}, 2, 2))::Array{Any,1}
└───       goto #12 if not true
2 ┄─ %3  = φ (#1 => 1, #11 => %24)::Int64
│    %4  = φ (#1 => 1, #11 => %25)::Int64
│    %5  = Base.add_int(1, %3)::Int64
│    %6  = Base.getfield(t, %5, true)::Union{Int64, String}
│    %7  = (isa)(%6, Int64)::Bool
└───       goto #4 if not %7
3 ── %9  = π (%6, Int64)
│          Base.arrayset(false, %1, %9, %3)::Array{Any,1}
└───       goto #7
4 ── %12 = (isa)(%6, String)::Bool
└───       goto #6 if not %12
5 ── %14 = π (%6, String)
│          Base.arrayset(false, %1, %14, %3)::Array{Any,1}
└───       goto #7
6 ──       Core.throw(ErrorException("fatal error in type inference (type bound)"))::Union{}
└───       $(Expr(:unreachable))::Union{}
7 ┄─ %19 = (%4 === 2)::Bool
└───       goto #9 if not %19
8 ──       goto #10
9 ── %22 = Base.add_int(%4, 1)::Int64
└───       goto #10
10 ┄ %24 = φ (#9 => %22)::Int64
│  

Not good. Key information is represented as integers, and when the compiler sees an integer it generally assumes it doesn't know its value.

- The compiler counts 1, infinity
- The compiler can match things but cannot do arithmetic or comparisons
- It's very good at knowing the types of function arguments

In [13]:
argtail(a, rest...) = rest
tupletail(t) = argtail(t...)

tupletail (generic function with 1 method)

In [14]:
tupletail((1,2,"hi"))

(2, "hi")

In [15]:
@code_typed tupletail((1,2,"hi"))

CodeInfo(
1 ─ %1 = (getfield)(t, 2)::Int64
│   %2 = (getfield)(t, 3)::String
│   %3 = Core.tuple(%1, %2)::Tuple{Int64,String}
└──      return %3
) => Tuple{Int64,String}

### Exercise

Write a type-inferable function to...

1. reverse a tuple
1. take every other element of a tuple
2. interleave the elements of two tuples

### Real-ish example: computing the shape of an indexing operation

In [ ]:
index_shape(a::Array, idxs) = ish(a, 1, idxs...)

ish(a, i,   ::Real...)         = ()
ish(a, i,   ::Colon,  rest...) = (size(a,i), ish(a,i+1,rest...)...)
ish(a, i, iv::Vector, rest...) = (length(iv), ish(a,i+1,rest...)...)
ish(a, i,   ::Real,   rest...) = ish(a,i+1,rest...)

In [ ]:
index_shape(rand(3,4,5), (1,:,[1,2]))

In [ ]:
index_shape(rand(3,4,5), (:,2,[1,2,1,2,1,2,1,2]))

## Trick 2: look up "trait" values and re-dispatch

### Functions of types

In [ ]:
widen(::Type{Float32}) = Float64

In [ ]:
widen(Float32)

In [ ]:
# We use this for type promotion
promote_type(Int64, Float64)

This can be used to compute attributes of types, then dispatch on those values.

In [ ]:
# Sample trait
abstract IteratorSize
immutable SizeUnknown <: IteratorSize end
immutable HasLength <: IteratorSize end
immutable HasShape <: IteratorSize end
immutable IsInfinite <: IteratorSize end

Now we can define a method that says which value of the trait a certain type has.

This is like using dispatch as a lookup table to find out properties of a combination of values.

In [ ]:
iteratorsize{T<:AbstractArray}(::Type{T}) = HasShape()

iteratorsize{I1,I2}(::Type{Zip2{I1,I2}}) = zip_iteratorsize(iteratorsize(I1),iteratorsize(I2))

zip_iteratorsize(a, b) = SizeUnknown()
zip_iteratorsize{T}(isz::T, ::T) = isz
zip_iteratorsize(::HasLength, ::HasShape) = HasLength()
zip_iteratorsize(::HasShape, ::HasLength) = HasLength()

In [ ]:
# `collect` gives you all the elements from an iterator as an array
collec(itr) = _collec(itr, eltype(itr), Base.iteratorsize(itr))

In [ ]:
function _collec(itr, T, ::Base.HasLength)
    a = Array{T,1}(length(itr))
    i = 0
    for x in itr
        a[i+=1] = x
    end
    return a
end

In [ ]:
function _collec(itr, T, ::Base.SizeUnknown)
    a = Array{T,1}(0)
    for x in itr
        push!(a, x)
    end
    return a
end